In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
# Load the pre-trained tokenizer for text processing
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# Load the pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

pytorch_model.bin:  74%|#######3  | 493M/669M [00:00<?, ?B/s]

In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [7]:
# Displaying the star number of the comment
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

4

In [8]:
# Fetch and parse Yelp reviews (exemple): Sends a GET request to the specified Yelp page, parses the HTML using BeautifulSoup,
# Finds all <p> elements with class names matching "comment" using a regex, and extracts the text content of these elements into a list.
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [9]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

In [10]:
import numpy as np
import pandas as pd

In [11]:
# Create a DataFrame from the extracted reviews
df = pd.DataFrame(np.array(reviews), columns=['review'])
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [12]:
# Define a function to calculate the sentiment score
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [13]:
# Exemple
sentiment_score(df['review'].iloc[1])

4

In [14]:
# Results
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:500]))

In [15]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,5
1,Six of us met here for breakfast before our wa...,4
2,We came for brunch and they ran out of seven s...,2
3,The food was delicious. The ricotta pancakes w...,4
4,Great place with delicious food and friendly s...,5
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,Great food amazing coffee and tea. Short walk ...,5
7,We came for brunch twice in our week-long visi...,4
8,"Great service, lovely location, and really ama...",5
9,It was ok. Had coffee with my friends. I'm new...,3


In [16]:
df['review'].iloc[6]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'